In [ ]:
import numpy as np
import cvxopt as cvx
import matplotlib.pyplot as plt
%matplotlib inline

from cvxopt import matrix, solvers
from scipy.integrate import odeint

In [ ]:
def ode1(x, tt, A, c):
    return np.dot(A, x)


def ode2(y, tt1, A, b):
    return np.dot(A, y) + b

In [ ]:
def createG(N):    
    G = np.eye(int(N), dtype=int).astype('float')
    G_neg = (np.eye(int(N), dtype=int) * -1).astype('float')

    G_res = matrix(np.concatenate((G, G_neg), axis=0))
    return G_res

def toQudratic(H, B, g, B0, c, D):
    An = np.dot(H, B)
    bn = g - np.dot(H, B0)
    cn = np.dot(B.transpose(), c + np.dot(D, B0))
    Dn = np.dot(np.dot(B.transpose(), D), B)
    h = list(np.linspace(1, 1, num=60))

    return An, bn, cn, Dn, h

In [ ]:
def createB(A, n, ode):
    B = np.random.random((len(A), int(n)))
    for i in range(int(n)):
        B[:, i] = ode[int(n) - i - 1]

    return B

In [ ]:
def optimalControl(D, A, b, c, x, H, g, t0, t1):
    n = 30
    y = np.zeros(len(A))
    Δ = (t1 - t0) / n
    tj = [t0 + i * Δ for i in range(1, int(n) + 1)]
    tj0 = tj[0]

    B0 = odeint(ode1, x, [t0, t1], args=(A, b))[1]
    y1 = odeint(ode2, y, [t0, tj0], args=(A, b))[1]
    ode = odeint(ode2, y1, tj, args=(A, b))

    B = createB(A, n, ode)
    An, bn, cn, Dn, h = toQudratic(H, B, g, B0, c, D)
    G = createG(n)

    u = solvers.qp(matrix(Dn), matrix(cn), G,
                   matrix(h), matrix(An), matrix(bn))['x']

    for i in range(0, int(n)):
        tj[i] -= Δ

    plt.figure(1)
    plt.plot(tj,u,color='green')
    plt.show()

In [ ]:
def example5():
    A = np.array([[-1, 1, 0, 0],
                [1, 0, -1, 0],
                [0, 1, 0, 1],
                [0, 0, 1, 0]])

    D = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

    b = np.array([0, 1, -1, 0])
    c = np.array([0, 0, 0, 0])

    x = [-1, -1, -1, -1]

    H = np.array([[0, 1, 1, 0],
             [1, -1, 0, -1]])

    g = np.array([-72.7564, -207.8494])
    t0 = 0.
    t1 = 8.

    optimalControl(D, A, b, c, x, H, g, t0, t1)

example5()